In [1]:
import tensorflow as tf

import numpy as np
import pandas as pd
import random as python_random

from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Lambda, BatchNormalization, Activation
from tensorflow.keras.optimizers import RMSprop,Adam
from tensorflow.keras import backend as K

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve,auc,roc_auc_score
from imblearn.under_sampling import RandomUnderSampler

### label

In [2]:
y = pd.read_csv("../cls_labelWithfea.csv")
print(len(y.loc[y["label"]==0]))
print(len(y.loc[y["label"]==1]))
y = y["label"]
y.head()

32827
2418


0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: label, dtype: float64

### input

In [3]:
# x = pd.read_csv("dis_fea_mat_noActi.csv",index_col=0)
x = pd.read_csv("../dis_fea_mat.csv",index_col=0)
print(x.shape)
x.tail()

(35245, 644)


,0,1,2,3,4,5,6,7,8,9,...,312.1,313.1,314.1,315.1,316.1,317.1,318.1,319.1,320.1,321.1
44542,12.516695,3.429878,56.012541,0.000000,0.000000,0.884564,20.813841,0.000000,0.000000,7.298384,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44544,12.516695,3.429878,56.012541,0.000000,0.000000,0.884564,20.813841,0.000000,0.000000,7.298384,...,0.0,17.58734,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44545,41.173339,13.719514,66.142468,5.570439,3.380668,17.691285,70.767060,4.527233,3.374051,13.933278,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44547,41.173339,13.719514,66.142468,5.570439,3.380668,17.691285,70.767060,4.527233,3.374051,13.933278,...,0.0,17.58734,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44549,8.234668,2.400915,20.855733,19.806004,8.451669,13.268464,15.818519,0.905447,2.530538,9.952341,...,0.0,17.58734,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### model---only symptom input

In [4]:
from tensorflow.keras.models import Sequential

x_arr = x.values
y_arr = y.values

i=1
cvacc=[]
cvauc=[]

seed = 22

import os
os.environ['PYTHONHASHSEED'] = '0'
# np.random.seed(seed)
python_random.seed(seed)
tf.random.set_seed(seed)


rus = RandomUnderSampler(sampling_strategy=1)

rus_x,rus_y = rus.fit_resample(X=x_arr,y=y_arr)
print('rus_y的长度:',len(rus_y))


"""10-fold"""
Kfold = StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)
for train,test in Kfold.split(X=rus_x,y=rus_y):
    print("--------------Kfold: {} iter".format(i))
    i+=1
    rms = RMSprop()
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss',patience=2)
    ##################################################################################################
    model = Sequential()
    model.add(Dense(32, activation='relu', input_dim=644))
    model.add(Dropout(0.1))
#     model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='sigmoid'))
    
    """L2"""
    for layer in model.layers:
        if hasattr(layer,'kernel_regularizer'):
            layer.kernel_regularizer = tf.keras.regularizers.l2(0.001)
    ##################################################################################################
    model.compile(
            loss=tf.keras.losses.BinaryCrossentropy(),
            optimizer=rms,
            metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.AUC()]
    )
    
    # fit()中 shuffle=True
    model.fit(rus_x[train],rus_y[train],
              #class_weight={0:1,1:1},
              batch_size=None,#defalut 32
              epochs=100,
              callbacks=callback,
              shuffle=True)
    
    scores = model.evaluate(rus_x[test],rus_y[test],
                            verbose=1,
                            batch_size=None,
                            callbacks=callback)
    
    cvacc.append(scores[1] * 100)
    cvauc.append(scores[2] * 100)
    
    print(" %s: %.2f%%\t ------ %s: %.2f%%\t " % 
           (model.metrics_names[1],scores[1]*100,model.metrics_names[2],scores[2]*100))
    
#print("ave_auc_keras: %.2f%% (+/- %.2f%%)\t ave_topk_pre: %.2f%% (+/- %.2f%%)\t ave_auc:%.2f%% (+/- %.2f%%)" % 
print("ave_acc: %.2f%% (+/- %.2f%%)\t ------ ave_auc: %.2f%% (+/- %.2f%%)\t " % 
      (np.mean(cvacc), np.std(cvacc),np.mean(cvauc), np.std(cvauc)))

print(model.summary())

rus_y的长度: 4836
--------------Kfold: 1 iter
Epoch 1/100
136/136 [==============================] - 0s 2ms/step - loss: 7.3176 - binary_accuracy: 0.5358 - auc: 0.5572
Epoch 2/100
136/136 [==============================] - 0s 1ms/step - loss: 2.6089 - binary_accuracy: 0.5678 - auc: 0.5921
Epoch 3/100
136/136 [==============================] - 0s 1ms/step - loss: 2.2240 - binary_accuracy: 0.5758 - auc: 0.6019
Epoch 4/100
136/136 [==============================] - 0s 1ms/step - loss: 1.7279 - binary_accuracy: 0.5742 - auc: 0.6091
Epoch 5/100
136/136 [==============================] - 0s 1ms/step - loss: 1.2833 - binary_accuracy: 0.5767 - auc: 0.6199
Epoch 6/100
136/136 [==============================] - 0s 1ms/step - loss: 1.0125 - binary_accuracy: 0.5974 - auc: 0.6532
Epoch 7/100
136/136 [==============================] - 0s 1ms/step - loss: 0.9394 - binary_accuracy: 0.5915 - auc: 0.6503
Epoch 8/100
136/136 [==============================] - 0s 1ms/step - loss: 0.9154 - binary_accuracy: 0.

137/137 [==============================] - 0s 2ms/step - loss: 7.9837 - binary_accuracy: 0.5206 - auc_6: 0.5361
Epoch 2/100
137/137 [==============================] - 0s 1ms/step - loss: 2.7906 - binary_accuracy: 0.5573 - auc_6: 0.5852
Epoch 3/100
137/137 [==============================] - 0s 1ms/step - loss: 1.8237 - binary_accuracy: 0.5762 - auc_6: 0.6158
Epoch 4/100
137/137 [==============================] - 0s 1ms/step - loss: 1.1611 - binary_accuracy: 0.5840 - auc_6: 0.6291
Epoch 5/100
137/137 [==============================] - 0s 1ms/step - loss: 1.0448 - binary_accuracy: 0.5847 - auc_6: 0.6339
Epoch 6/100
137/137 [==============================] - 0s 1ms/step - loss: 0.8972 - binary_accuracy: 0.5934 - auc_6: 0.6431
Epoch 7/100
137/137 [==============================] - 0s 1ms/step - loss: 0.7923 - binary_accuracy: 0.5989 - auc_6: 0.6511
Epoch 8/100
137/137 [==============================] - 0s 1ms/step - loss: 0.7111 - binary_accuracy: 0.6138 - auc_6: 0.6741
Epoch 9/100
137/137 

#### results:
32-32-16-1/patience=2:
    ave_acc: 65.38% (+/- 2.20%)	 ------ ave_auc: 71.63% (+/- 1.82%)

### model---sym+mesh

In [6]:
x = pd.read_csv("../sym_mesh_concat.csv",index_col=0)
print(x.shape)
x.tail()

(35245, 692)


,0,1,2,3,4,5,6,7,8,9,...,38.1.1,39.1.1,40.1.1,41.1.1,42.1.1,43.1.1,44.1.1,45.1.1,46.1.1,47.1.1
35240,12.516695,3.429878,56.012541,0.000000,0.000000,0.884564,20.813841,0.000000,0.000000,7.298384,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35241,12.516695,3.429878,56.012541,0.000000,0.000000,0.884564,20.813841,0.000000,0.000000,7.298384,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
35242,41.173339,13.719514,66.142468,5.570439,3.380668,17.691285,70.767060,4.527233,3.374051,13.933278,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35243,41.173339,13.719514,66.142468,5.570439,3.380668,17.691285,70.767060,4.527233,3.374051,13.933278,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
35244,8.234668,2.400915,20.855733,19.806004,8.451669,13.268464,15.818519,0.905447,2.530538,9.952341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [13]:
from tensorflow.keras.models import Sequential

x_arr = x.values
y_arr = y.values

i=1
cvacc=[]
cvauc=[]

seed = 22

import os
os.environ['PYTHONHASHSEED'] = '0'
# np.random.seed(seed)
python_random.seed(seed)
tf.random.set_seed(seed)


rus = RandomUnderSampler(sampling_strategy=1)

rus_x,rus_y = rus.fit_resample(X=x_arr,y=y_arr)
print('rus_y的长度:',len(rus_y))


"""10-fold"""
Kfold = StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)
for train,test in Kfold.split(X=rus_x,y=rus_y):
    print("--------------Kfold: {} iter".format(i))
    i+=1
    rms = RMSprop()
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss',patience=2)
    ##################################################################################################
    model = Sequential()
    model.add(Dense(128, activation='relu', input_dim=692))
    model.add(Dropout(0.1))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='sigmoid'))
    
    """L2"""
    for layer in model.layers:
        if hasattr(layer,'kernel_regularizer'):
            layer.kernel_regularizer = tf.keras.regularizers.l2(0.001)
    ##################################################################################################
    model.compile(
            loss=tf.keras.losses.BinaryCrossentropy(),
            optimizer=rms,
            metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.AUC()]
    )
    
    # fit()中 shuffle=True
    model.fit(rus_x[train],rus_y[train],
              #class_weight={0:1,1:1},
              batch_size=None,
              epochs=100,
              callbacks=callback,
              shuffle=True)
    
    scores = model.evaluate(rus_x[test],rus_y[test],
                            verbose=1,
                            batch_size=None,
                            callbacks=callback)
    
    cvacc.append(scores[1] * 100)
    cvauc.append(scores[2] * 100)
    
    print(" %s: %.2f%%\t ------ %s: %.2f%%\t " % 
           (model.metrics_names[1],scores[1]*100,model.metrics_names[2],scores[2]*100))
    
#print("ave_auc_keras: %.2f%% (+/- %.2f%%)\t ave_topk_pre: %.2f%% (+/- %.2f%%)\t ave_auc:%.2f%% (+/- %.2f%%)" % 
print("ave_acc: %.2f%% (+/- %.2f%%)\t ------ ave_auc: %.2f%% (+/- %.2f%%)\t " % 
      (np.mean(cvacc), np.std(cvacc),np.mean(cvauc), np.std(cvauc)))

print(model.summary())

rus_y的长度: 4836
--------------Kfold: 1 iter
Epoch 1/100
136/136 [==============================] - 0s 2ms/step - loss: 3.5687 - binary_accuracy: 0.5186 - auc_62: 0.5399
Epoch 2/100
136/136 [==============================] - 0s 2ms/step - loss: 1.3294 - binary_accuracy: 0.5528 - auc_62: 0.5696
Epoch 3/100
136/136 [==============================] - 0s 2ms/step - loss: 0.9977 - binary_accuracy: 0.5634 - auc_62: 0.5752
Epoch 4/100
136/136 [==============================] - 0s 2ms/step - loss: 0.8381 - binary_accuracy: 0.5641 - auc_62: 0.5863
Epoch 5/100
136/136 [==============================] - 0s 2ms/step - loss: 0.7286 - binary_accuracy: 0.5850 - auc_62: 0.6142
Epoch 6/100
136/136 [==============================] - 0s 2ms/step - loss: 0.7087 - binary_accuracy: 0.5823 - auc_62: 0.6239
Epoch 7/100
136/136 [==============================] - 0s 2ms/step - loss: 0.7081 - binary_accuracy: 0.5924 - auc_62: 0.6375
Epoch 8/100
136/136 [==============================] - 0s 2ms/step - loss: 0.6848 

136/136 [==============================] - 0s 2ms/step - loss: 0.5347 - binary_accuracy: 0.7530 - auc_67: 0.8413
Epoch 27/100
16/16 [==============================] - 0s 2ms/step - loss: 0.6458 - binary_accuracy: 0.6860 - auc_67: 0.7684
 binary_accuracy: 68.60%	 ------ auc_67: 76.84%	 
--------------Kfold: 7 iter
Epoch 1/100
137/137 [==============================] - 0s 2ms/step - loss: 3.3689 - binary_accuracy: 0.5288 - auc_68: 0.5470
Epoch 2/100
137/137 [==============================] - 0s 2ms/step - loss: 1.6739 - binary_accuracy: 0.5284 - auc_68: 0.5417
Epoch 3/100
137/137 [==============================] - 0s 2ms/step - loss: 1.0851 - binary_accuracy: 0.5376 - auc_68: 0.5414
Epoch 4/100
137/137 [==============================] - 0s 2ms/step - loss: 0.8918 - binary_accuracy: 0.5376 - auc_68: 0.5478
Epoch 5/100
137/137 [==============================] - 0s 2ms/step - loss: 0.7558 - binary_accuracy: 0.5555 - auc_68: 0.5672
Epoch 6/100
137/137 [==============================] - 0s 2m

#### results
    1.64-32-16-1/patience=2:        ave_acc: 64.85% (+/- 3.89%)	 ------ ave_auc: 71.16% (+/- 4.31%)
    2.128-64-64-32-16-1/patience=2:   ave_acc: 66.54% (+/- 2.32%)	 ------ ave_auc: 73.46% (+/- 2.96%)